In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("EL Nino (5-D).csv")         #Importing El- nino dataset
df.head()

,Observation,Year,Month,Day,Date,Latitude,Longitude,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,4060,93,5,9,930509,-0.02,-109.96,-2.1,2.1,81.2,26.80,27.02
1,4061,93,5,10,930510,-0.02,-109.96,-3.4,1.4,84.2,26.95,26.91
2,4062,93,5,11,930511,-0.02,-109.96,-3.8,2.2,84.9,26.98,26.78
3,4063,93,5,12,930512,-0.02,-109.96,-3.0,1.5,86.9,26.93,26.74
4,4064,93,5,13,930513,-0.02,-109.96,-4.5,1.9,87.6,27.01,26.82


In [ ]:
data = df.drop(df.columns[0:7],axis=1)            #Removing spatio-temporial attributes
data.head()

,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,-2.1,2.1,81.2,26.80,27.02
1,-3.4,1.4,84.2,26.95,26.91
2,-3.8,2.2,84.9,26.98,26.78
3,-3.0,1.5,86.9,26.93,26.74
4,-4.5,1.9,87.6,27.01,26.82


In [ ]:
data.shape
data.columns
data

,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,-2.1,2.1,81.2,26.80,27.02
1,-3.4,1.4,84.2,26.95,26.91
2,-3.8,2.2,84.9,26.98,26.78
3,-3.0,1.5,86.9,26.93,26.74
4,-4.5,1.9,87.6,27.01,26.82
...,...,...,...,...,...
93930,-6.8,-5.3,81.3,27.52,28.17
93931,-5.1,-0.4,94.1,26.04,28.14
93932,-4.3,-3.3,93.2,25.80,27.87
93933,-6.1,-4.8,81.3,27.17,27.93


In [ ]:
#Shuffling the dataset row-vise

shuffled_data = data.sample(frac=1, random_state=42)      # frac=1 means you want to sample the entire DataFrame

# Reset the index if you want to reset the index after shuffling
shuffled_data.reset_index(drop=True, inplace=True)

S = shuffled_data.sample(frac=0.000595*2, random_state=42)                 # Total dataset S for later use
S_complement=shuffled_data.drop(S.index)
sprime = S_complement.sample(frac=0.000595*2,random_state=42)

S_ = S.copy()
s1 = S_.sample(frac=0.5, random_state=42)     #sampling half dataset for training
s2 = S_.drop(s1.index)

s1.reset_index(drop=True, inplace=True)
s2.reset_index(drop=True, inplace=True)
sprime.reset_index(drop=True, inplace=True)

print(s1.shape)
print(s2.shape)
print(sprime.shape)

len_s1 = s1.shape[0]
len_s2 = s2.shape[0]
len_sprime = sprime.shape[0]

cov_s1 = s1.cov(ddof=1)         #Covariance matrix of s1

s1_matrix = s1.to_numpy()
s2_matrix = s2.to_numpy()
sprime_matrix = sprime.to_numpy()
cov_s1_matrix0 = cov_s1.to_numpy()
len_s1 , len_s2 , len_sprime

(56, 5)
(56, 5)
(112, 5)


(56, 56, 112)

$\Huge{\text{Required Functions for Algorithm}}$

# $ \textbf{Equation 5:} \\
\left\{\begin{array}{l}
p\left(x_{j} \mid \omega_{i}\right)=\frac{1}{(2 \pi)^{k / 2}\left|\Sigma_{x_{i}}\right|^{1 / 2}} \exp \left(-\frac{1}{2}\left(x_{j}-x_{i}\right)^{T} \Sigma_{x_{i}}^{-1}\left(x_{j}-x_{i}\right)\right) \\
p\left(x_{i} \mid \omega_{i}\right)=0, \quad i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\
\quad i=1,2, \ldots\left|S_{1}\right|
\end{array}\right.
$

In [ ]:
k = 5        # Dimensionality of s1

def small_p(xi, xj, cov):

    cov_1 = cov

    det_cov = np.linalg.det(cov)
    epsilon = 1e-2
    det_cov = max(det_cov, epsilon)

    if det_cov == 0.01:
        alpha = 1e-2
        cov_1 = cov_1 + alpha * np.identity(5)

    a = 1 / (((2 * np.pi) ** (k / 2)) * det_cov ** 0.5)
    b = -0.5 * ((xj - xi).T @ np.linalg.inv(cov_1) @ (xj - xi))

    if b > 1000:
        print('b value', b)
    b = min(b, 50)
    p = a * np.exp(b)

    return p

# $ \textbf{Equation 4:} \\
\begin{cases}P\left(\omega_{i} \mid x_{j}\right)=\frac{p\left(x_{j} \mid \omega_{i}\right)}{\sum_{t=1}^{\left|S_{1}\right|} p\left(x_{j} \mid \omega_{t}\right)} & i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\ P\left(\omega_{i} \mid x_{i}\right)=0 & i=1,2, \ldots\left|S_{1}\right|\end{cases}
$

In [ ]:
def capital_p(i, j, cov_mat):
    summation = 0
    xi = s1_matrix[i, :].reshape(-1, 1)
    xj = s1_matrix[j, :].reshape(-1, 1)
    if i != j:
        for k in range(len_s1):
            if k != j:
                xk = s1_matrix[k, :].reshape(-1, 1)
                summation += small_p(xk,xj, cov_mat[k])

    # Avoid division by zero
    if summation != 0:
        P = small_p(xi,xj,cov_mat[i]) / summation

        return  P

    return 0

# $ \textbf{Equation 3:} \\
\Sigma_{x_{i}}=\frac{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)\left(x_{j}-x_{i}\right)\left(x_{j}-x_{i}\right)^{T}}{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)}
$

In [ ]:
def cov_xi(i,cov_mat):
    denominator = 0
    numerator = 0

    xi = s1_matrix[i, :].reshape(-1, 1)
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)

        P = capital_p(i, j, cov_mat)

        denominator += P
        numerator += P * ((xj - xi) @ (xj - xi).T)

    return numerator / denominator

In [ ]:
def L(cov_mat):
    L_new=0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        summation=0
        for i in range(len_s1):
            if i!=j:
                xi = s1_matrix[i, :].reshape(-1, 1)
                summation+=small_p(xi,xj,cov_mat[i])

        summation=summation/(len_s1-1)
        L_new+=np.log(summation)

    return L_new[0][0]

# $ \textbf{Equation 2:}$
$ \ L=\sum_{x_{j} \in S_{1}} \log \left[\sum_{x_{i} \in S_{1} \wedge i \neq j} \frac{1}{\left|S_{1}\right|-1} G\left(\Sigma_{x_{i}}, x_{j}-x_{i}\right)\right]
$

$\Huge{\text{Algorithm 1}}$

In [ ]:
def Algorithm1(cov_mat):
    t = 0
    phi = 0.01
    L_prev = -250

    while t < 100:

        temp = []

        for i in range(len_s1):
            temp.append(cov_xi(i,cov_mat))

        cov_mat = temp.copy()

        for i in range(len_s1):
            if np.linalg.det(cov_mat[i]) <= 0:
                alpha = 1e-2
                cov_mat[i] = cov_mat[i] + alpha * np.identity(5)

        L_new = L(cov_mat)

        #print(((L_new-L_prev)/L_prev))
        #print('L_prev=',L_prev)
        #print('L_new=',L_new)
        #print('t=',t)
        #print('-'*50)

        if abs((L_new-L_prev)/L_prev) < phi:
            break

        else:
            L_prev=L_new
            t+=1

    return cov_mat

In [ ]:
#initial guess
cov_mat_initial=[]
for i in range(len_s1):
    xi = s1_matrix[i, :].reshape(-1, 1)
    mat=0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        mat+=(xj-xi)@(xj-xi).T
    cov_mat_initial.append(mat/(len_s1-1))

In [ ]:
cov_mat= Algorithm1(cov_mat_initial)

In [ ]:
cov_mat

[array([[ 1.06927506,  1.0133872 , -0.53900991, -0.00240916,  0.11927105],
        [ 1.0133872 ,  1.07059102, -0.11241228, -0.2256428 , -0.01524373],
        [-0.53900991, -0.11241228,  1.71258905, -0.80655165, -0.52404294],
        [-0.00240916, -0.2256428 , -0.80655165,  0.45284399,  0.25980747],
        [ 0.11927105, -0.01524373, -0.52404294,  0.25980747,  0.16267204]]),
 array([[  7.09834506,   2.79943942, -11.48010439,   1.90813553,
           2.34210683],
        [  2.79943942,   6.81750472,   8.96579847,  -1.49841701,
          -1.33609846],
        [-11.48010439,   8.96579847,  50.43343454,  -8.40200412,
          -9.12471809],
        [  1.90813553,  -1.49841701,  -8.40200412,   1.39974388,
           1.51988055],
        [  2.34210683,  -1.33609846,  -9.12471809,   1.51988055,
           1.66656256]]),
 array([[ 3.10416274,  4.15498335,  1.61729206, -0.16178686,  0.21344044],
        [ 4.15498335,  5.72328595,  3.49151615, -0.48416533,  0.07021712],
        [ 1.61729206,  3.4

$\Huge{\text{Algorithm 2}}$

In [ ]:
def F(T):
    s = 0
    for i in range(len_s1):
        xi = s1_matrix[i, :].reshape(-1, 1)
        s += small_p(xi,T,cov_mat[i])
    f = np.log(s/len_s1)
    return f

def EstVar(estsize,beta):
    Est = []
    for t in range(1,estsize):
        R = s2.sample(n=len(s2), replace=True, random_state=42)
        R = R.to_numpy()
        F_R = []
        for i in range(len_s2):
            T = R[i, :].reshape(-1, 1)
            F_R.append(F(T))

        var_F_R = np.var(np.array(F_R))
        Est.append((len_s2/(len_s2-1))*var_F_R)

    V =  np.percentile(Est, estsize*(1-beta))
    var_delta = (len_sprime + ((len_sprime**2)/len_s2))*V
    return var_delta

$\Huge{\text{Algorithm 3}}$

In [ ]:
from scipy.stats import norm

def critical_value(p,stepsize):
    estsize = 100
    M = int(np.ceil((p/stepsize)-1))
    C = []
    for i in range(1,M+1):
        #print('step number:',i,'/',M+1)
        alpha_i = i*stepsize
        beta_i = p-alpha_i
        var_delta = EstVar(estsize,beta_i)

        mean = 0

        std_dev = np.sqrt(var_delta)
        c = norm.ppf(alpha_i, loc = mean, scale = std_dev)
        C.append(c)

    C_max = max(C)

    return C_max

# $\textbf{Equation 7:}$
 $
\begin{aligned}
\delta= & \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S^{\prime}\right)-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S_{2}\right) \\
= & \log \left\{\prod_{y \in S^{\prime}} \mathscr{K}_{S_{1}}(y)\right\}-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\prod_{y \in S_{2}} \mathscr{K}_{S_{1}}(y)\right\} \\
= & \sum_{y \in S^{\prime}} \log \sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right) \\
& -\sum_{y \in S_{2}} \frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right)\right\}
\end{aligned}
$

In [ ]:
def delta():

    a = 0

    for j in range(len_sprime):
        y = sprime_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        a += np.log(summation/len_s1)

    b = 0

    for j in range(len_s2):
        y = s2_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        b += np.log(summation/len_s1)

    b = b*len_sprime/len_s2

    delta = a-b

    return delta

In [ ]:
sprime_matrix.shape

(112, 5)

$\Huge{\text{Change Detection Decision}}$

In [ ]:
p = 0.08/2
stepsize = 0.002

Delta = delta()

C_max= critical_value(p,stepsize)

if Delta < C_max:
    print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

else:
    print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 


$\Huge{\text{Two way test}}$
$\text{}$

In [ ]:
S_ = sprime.copy()
s1 = S_.sample(frac=0.5, random_state=42)     #sampling half dataset for training
s2 = S_.drop(s1.index)
sprime = S.copy()

s1.reset_index(drop=True, inplace=True)
s2.reset_index(drop=True, inplace=True)
sprime.reset_index(drop=True, inplace=True)

print(s1.shape)
print(s2.shape)
print(sprime.shape)
len_s1= s1.shape[0]
len_s2= s2.shape[0]
len_sprime = sprime.shape[0]

s1_matrix = s1.to_numpy()
s2_matrix = s2.to_numpy()
sprime_matrix=sprime.to_numpy()

len_s1 , len_s2 , len_sprime

(56, 5)
(56, 5)
(112, 5)


(56, 56, 112)

In [ ]:
#initial guess
cov_mat_initial=[]
for i in range(len_s1):
    xi = s1_matrix[i, :].reshape(-1, 1)
    mat=0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        mat+=(xj-xi)@(xj-xi).T
    cov_mat_initial.append(mat/(len_s1-1))

In [ ]:
#ALGORITHM 1

cov_mat=Algorithm1(cov_mat_initial)


In [ ]:
cov_mat

[array([[ 5.1905536 ,  3.81036671, -7.87740322,  1.51680742,  1.60437001],
        [ 3.81036671,  2.91536142, -5.82711209,  1.19700578,  1.27531067],
        [-7.87740322, -5.82711209, 11.99886484, -2.33267955, -2.47031176],
        [ 1.51680742,  1.19700578, -2.33267955,  0.5185261 ,  0.54516897],
        [ 1.60437001,  1.27531067, -2.47031176,  0.54516897,  0.59517201]]),
 array([[ 1.54329215, -2.6459444 ,  0.36166152, -0.9250042 , -0.93380296],
        [-2.6459444 ,  4.86954811, -0.72      ,  1.48707379,  1.47995058],
        [ 0.36166152, -0.72      ,  0.11473477, -0.18712096, -0.18252035],
        [-0.9250042 ,  1.48707379, -0.18712096,  0.58373942,  0.59560168],
        [-0.93380296,  1.47995058, -0.18252035,  0.59560168,  0.60899466]]),
 array([[ 2.05372061, -1.22121237,  2.17054488, -0.204418  ,  0.05752365],
        [-1.22121237,  0.77317102, -1.26057926,  0.05081347, -0.09102316],
        [ 2.17054488, -1.26057926,  2.44670243, -0.157334  ,  0.11975604],
        [-0.204418  ,

In [ ]:
#Change detection
p=0.08/2
stepsize=0.002

Delta = delta()
C_max= critical_value(p,stepsize)

if Delta<C_max:
    print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

else:
    print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 


$\Huge{\text{Experiment 1}}$
$\text{}$

$\text{We have run the experiment 1 for 20 instances because it's time complexity is high.}$
$\text{still we are getting expected results.}$

In [ ]:
#Running for 20 instances

fp=0

for i in range(20):
    print('itr=',i)
    N = data.sample(frac = 0.0023846, replace = True, random_state = 42)
    S = N.sample(frac = 0.5,random_state = 42)
    sprime = N.drop(S.index)
    S.reset_index(drop = True, inplace = True)
    sprime.reset_index(drop = True, inplace = True)

    S_ = S.copy()
    s1 = S_.sample(frac = 0.5, random_state = 42)
    s2 = S_.drop(s1.index)
    s1.reset_index(drop = True, inplace = True)
    s2.reset_index(drop = True, inplace = True)

    len_s1 = s1.shape[0]
    len_s2 = s2.shape[0]
    len_sprime = sprime.shape[0]

    s1_matrix = s1.to_numpy()
    s2_matrix = s2.to_numpy()
    sprime_matrix = sprime.to_numpy()

    #initial guess
    cov_mat_initial = []
    for i in range(len_s1):
        xi = s1_matrix[i, :].reshape(-1, 1)
        mat = 0
        for j in range(len_s1):
            xj = s1_matrix[j, :].reshape(-1, 1)
            mat += (xj-xi)@(xj-xi).T
        cov_mat_initial.append(mat/(len_s1-1))

    cov_mat = Algorithm1(cov_mat_initial)

    #Change detection
    p = 0.08/2
    stepsize = 0.002

    Delta = delta()
    C_max = critical_value(p,stepsize)

    if Delta < C_max:
        fp += 1
        print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

    else:
        print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

        S_ = sprime.copy()
        s1 = S_.sample(frac=0.5, random_state=42)     # Sampling half dataset for training
        s2 = S_.drop(s1.index)
        sprime = S.copy()

        s1.reset_index(drop=True, inplace=True)
        s2.reset_index(drop=True, inplace=True)
        sprime.reset_index(drop=True, inplace=True)

        len_s1= s1.shape[0]
        len_s2= s2.shape[0]
        len_sprime=sprime.shape[0]

        s1_matrix = s1.to_numpy()
        s2_matrix = s2.to_numpy()
        sprime_matrix=sprime.to_numpy()

        #initial guess
        cov_mat_initial=[]
        for i in range(len_s1):
            xi = s1_matrix[i, :].reshape(-1, 1)
            mat=0
            for j in range(len_s1):
                xj = s1_matrix[j, :].reshape(-1, 1)
                mat+=(xj-xi)@(xj-xi).T
            cov_mat_initial.append(mat/(len_s1-1))

        cov_mat = Algorithm1(cov_mat_initial)

        #Change detection

        p=0.08/2
        stepsize=0.002

        Delta = delta()
        C_max= critical_value(p,stepsize)

        if Delta<C_max:
            fp+=1
            print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S')

        print('fp:',fp)
        print('')

print('FALSE POSITIVE RATE:', fp/20)

itr= 0
fp: 0

itr= 1
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 2
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 3
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 4
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 5
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 6
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 7
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 8
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 9
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 10
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 11
THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S 
fp: 0

itr= 12
THERE IS